In [1]:
!pip install duckdb==1.0.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 36.0 MB/s eta 0:00:0000:0100:01


In [27]:
import duckdb

# create a connection to a file called 'demo_duckdb.db'
con = duckdb.connect('demo_duckdb.duckdb')

In [28]:
con.sql(
    """
    SELECT * FROM read_json("glossary.json")
    """
).df()

,glossary
0,"{'title': 'example glossary', 'GlossDiv': {'ti..."


In [29]:
con.sql(
    """
    SELECT glossary.GlossDiv.GlossList.GlossEntry.ID FROM read_json("glossary.json")
    """
).df()

,ID
0,SGML


In [34]:
con.sql(
    """
    SELECT * FROM read_json('https://support.oneskyapp.com/hc/en-us/article_attachments/202761627')
    """
).df()

,fruit,size,color
0,Apple,Large,Red


In [36]:
con.sql(
    """
    SELECT * FROM read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet')
    """
).df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,1.0,1.72,1.0,N,186,79,2,17.70,1.00,0.5,0.00,0.00,1.0,22.70,2.5,0.0
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,1.0,1.80,1.0,N,140,236,1,10.00,3.50,0.5,3.75,0.00,1.0,18.75,2.5,0.0
2,1,2024-01-01 00:17:06,2024-01-01 00:35:01,1.0,4.70,1.0,N,236,79,1,23.30,3.50,0.5,3.00,0.00,1.0,31.30,2.5,0.0
3,1,2024-01-01 00:36:38,2024-01-01 00:44:56,1.0,1.40,1.0,N,79,211,1,10.00,3.50,0.5,2.00,0.00,1.0,17.00,2.5,0.0
4,1,2024-01-01 00:46:51,2024-01-01 00:52:57,1.0,0.80,1.0,N,211,148,1,7.90,3.50,0.5,3.20,0.00,1.0,16.10,2.5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2964619,2,2024-01-31 23:45:59,2024-01-31 23:54:36,NaN,3.18,NaN,None,107,263,0,15.77,0.00,0.5,2.00,0.00,1.0,21.77,NaN,NaN
2964620,1,2024-01-31 23:13:07,2024-01-31 23:27:52,NaN,4.00,NaN,None,114,236,0,18.40,1.00,0.5,2.34,0.00,1.0,25.74,NaN,NaN
2964621,2,2024-01-31 23:19:00,2024-01-31 23:38:00,NaN,3.33,NaN,None,211,25,0,19.97,0.00,0.5,0.00,0.00,1.0,23.97,NaN,NaN
2964622,2,2024-01-31 23:07:23,2024-01-31 23:25:14,NaN,3.06,NaN,None,107,13,0,23.88,0.00,0.5,5.58,0.00,1.0,33.46,NaN,NaN


In [37]:
%%time

con.sql(
    """
    CREATE TABLE yellow_tripdata_2024_01 AS
    SELECT * FROM read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet')
    """
)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: user 3.84 s, sys: 59.4 ms, total: 3.9 s
Wall time: 5.77 s


In [38]:
%%time

con.sql(
    """
    CREATE TABLE yellow_tripdata_2024_01_local AS
    SELECT * FROM read_parquet('yellow_tripdata_2024-01.parquet')
    """
)

CPU times: user 2.77 s, sys: 260 ms, total: 3.03 s
Wall time: 1.27 s


In [105]:
%%time

con.sql(
    """
    SELECT COUNT(*) FROM yellow_tripdata_2024_01
    """
)

CPU times: user 322 µs, sys: 101 µs, total: 423 µs
Wall time: 427 µs


┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│      2964624 │
└──────────────┘

In [118]:
%%time

con.sql(
    """
    SELECT 
        tpep_pickup_datetime::date AS date_, 
        SUM(total_amount) AS sum_,
        COUNT(*) AS count_,
        SUM(total_amount) / COUNT(*) AS avg_ 
    FROM 
        yellow_tripdata_2024_01
    WHERE
        tpep_pickup_datetime BETWEEN '2024-01-01' AND '2024-01-31'
    GROUP BY 1
    ORDER BY 1
    """
)

CPU times: user 650 µs, sys: 0 ns, total: 650 µs
Wall time: 656 µs


┌────────────┬────────────────────┬────────┬────────────────────┐
│   date_    │        sum_        │ count_ │        avg_        │
│    date    │       double       │ int64  │       double       │
├────────────┼────────────────────┼────────┼────────────────────┤
│ 2024-01-01 │  2442843.249999854 │  81013 │ 30.153719156183005 │
│ 2024-01-02 │  2282196.019999924 │  75519 │  30.22015678173604 │
│ 2024-01-03 │  2357588.479999926 │  82427 │ 28.602138619626164 │
│ 2024-01-04 │  2800511.059999929 │ 102901 │ 27.215586437448895 │
│ 2024-01-05 │ 2728672.5199999115 │ 103178 │   26.4462629630339 │
│ 2024-01-06 │  2436272.319999972 │  97117 │ 25.085951172297044 │
│ 2024-01-07 │  1898102.159999877 │  67543 │ 28.102129902430704 │
│ 2024-01-08 │  2216447.229999935 │  80034 │ 27.693820501286137 │
│ 2024-01-09 │ 2363882.4900000454 │  93962 │  25.15785626104218 │
│ 2024-01-10 │ 2550418.1299999813 │  95000 │  26.84650663157875 │
│     ·      │          ·         │    ·   │          ·         │
│     ·   

In [151]:
%%time

con.sql(
    """
    SELECT 
        tpep_pickup_datetime::date AS date_, 
        SUM(total_amount) AS sum_,
        COUNT(*) AS count_,
        SUM(total_amount) / COUNT(*) AS avg_ 
    FROM 
        yellow_tripdata_2024_01
    WHERE
        tpep_pickup_datetime BETWEEN '2024-01-01' AND '2024-01-31'
    GROUP BY 1
    ORDER BY 1
    """
).df()

CPU times: user 72.6 ms, sys: 0 ns, total: 72.6 ms
Wall time: 24.3 ms


,date_,sum_,count_,avg_
0,2024-01-01,2442843.25,81013,30.153719
1,2024-01-02,2282196.02,75519,30.220157
2,2024-01-03,2357588.48,82427,28.602139
3,2024-01-04,2800511.06,102901,27.215586
4,2024-01-05,2728672.52,103178,26.446263
5,2024-01-06,2436272.32,97117,25.085951
6,2024-01-07,1898102.16,67543,28.102130
7,2024-01-08,2216447.23,80034,27.693821
8,2024-01-09,2363882.49,93962,25.157856
9,2024-01-10,2550418.13,95000,26.846507


In [154]:
con.sql(
    """
    SELECT * FROM information_schema.columns
    WHERE table_name = 'yellow_tripdata_2024_01'
    """
)

┌───────────────┬──────────────┬──────────────────────┬───┬──────────────────────┬──────────────┬────────────────┐
│ table_catalog │ table_schema │      table_name      │ … │ generation_express…  │ is_updatable │ COLUMN_COMMENT │
│    varchar    │   varchar    │       varchar        │   │       varchar        │   boolean    │    varchar     │
├───────────────┼──────────────┼──────────────────────┼───┼──────────────────────┼──────────────┼────────────────┤
│ demo_duckdb   │ main         │ yellow_tripdata_20…  │ … │ NULL                 │ NULL         │ NULL           │
│ demo_duckdb   │ main         │ yellow_tripdata_20…  │ … │ NULL                 │ NULL         │ NULL           │
│ demo_duckdb   │ main         │ yellow_tripdata_20…  │ … │ NULL                 │ NULL         │ NULL           │
│ demo_duckdb   │ main         │ yellow_tripdata_20…  │ … │ NULL                 │ NULL         │ NULL           │
│ demo_duckdb   │ main         │ yellow_tripdata_20…  │ … │ NULL                

In [3]:
con.close()